In [30]:
import pandas as pd
import numpy as np
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split

C:\Users\evger\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [112]:
def fix_column(df, column):
    averages = {}
    amount = {}
    df[column] = df[column].fillna(0)
    
    regions = np.unique(df['Region'].values)
    for c in regions:
        averages[c] = 0
        amount[c] = 0
    fix_indexes = []
    
    for i in range(len(df)):
        region = df.loc[i, 'Region']
        value = df.loc[i, column]
        if value != 0:
            averages[region] += value
            amount[region] += 1
        else:
            fix_indexes.append(i)
    
    for reg in regions:
        averages[reg] /= amount[reg]
    
    for fix in fix_indexes:
        region = df.loc[fix, 'Region']
        value = averages[region]
        df.loc[fix, column] = value
        
    return df
        
def fix_columns(df, columns):
    print(columns)
    for column in columns:
        df = fix_column(df, column)
    return df
        

In [113]:
dataset = pd.read_csv('datasets/countries.csv')
# dataset = dataset.dropna()
# dataset = fix_column(dataset, 'Climate')
# dataset = fix_column(dataset, 'Agriculture')
# dataset = fix_column(dataset, 'Industry')
# dataset = fix_column(dataset, 'Service')
# dataset = fix_column(dataset, 'Birthrate')
# dataset = fix_column(dataset, 'Deathrate')
# dataset = fix_column(dataset, 'GDP ($ per capita)')
dataset = fix_columns(dataset, [item for item in dataset.columns.values if item not in ['Region', 'Country']])

print(dataset.dtypes)



dataset['Region'] = dataset.Region.apply(lambda x: x.replace('EUROPE', '1'))
dataset['Region'] = dataset.Region.apply(lambda x: x.replace('ASIA', '-1'))
dataset['Region'] = dataset.Region.apply(lambda x: x.replace('AMERICA', '-1'))
dataset['Region'] = dataset.Region.apply(lambda x: x.replace('AFRICA', '-1'))
dataset['Region'] = dataset.Region.apply(lambda x: x.replace('OCEANIA', '-1'))
dataset['Region'] = dataset['Region'].astype('float64')

y = dataset[['Region']]
X = dataset[[val for val in dataset.columns.values if val not in ['Region', 'Country']]]

['Population', 'Area (sq. mi.)', 'Pop. Density (per sq. mi.)', 'Coastline (coast/area ratio)', 'Net migration', 'Infant mortality (per 1000 births)', 'Literacy (%)', 'Phones (per 1000)', 'Arable (%)', 'Crops (%)', 'Other (%)', 'Climate', 'Birthrate', 'Deathrate', 'Agriculture', 'Industry', 'Service', 'GDP ($ per capita)']
Country                                object
Region                                 object
Population                              int64
Area (sq. mi.)                          int64
Pop. Density (per sq. mi.)            float64
Coastline (coast/area ratio)          float64
Net migration                         float64
Infant mortality (per 1000 births)    float64
Literacy (%)                          float64
Phones (per 1000)                     float64
Arable (%)                            float64
Crops (%)                             float64
Other (%)                             float64
Climate                               float64
Birthrate                       

In [109]:
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.3)
ada = AdaBoostClassifier(n_estimators=50, learning_rate=1.0)
ada.fit(train_x, train_y)
predicted = ada.predict(test_x)
# print(predicted)
# print(test_y.values)
# print(sum(predicted != test_y.values))
score = ada.score(test_x, test_y)
print(score)


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').